In [1]:
import requests
import time
import random
import pandas as pd
import os
import json
from tqdm import tqdm
import numpy as np
import math

In [2]:
cookies = {
    'TIKI_GUEST_TOKEN': '8jWSuIDBb2NGVzr6hsUZXpkP1FRin7lY',
    'TOKENS': '{%22access_token%22:%22eyXqm3C1WZOV4xJQKsTYcjgDrwzu6fh8%22}',
    'amp_99d374': 'eSc-_0HT1um7cb57E7dwA0...1enloc6a2.1enlpc1q6.b.i.t',
    'amp_99d374_tiki.vn': 'eSc-_0HT1um7cb57E7dwA0...1enloc6a2.1enlp8817.3.1.1',
    '_gcl_au': '1.1.559117409.1605974236',
    '_ants_utm_v2': '',
    '_pk_id.638735871.2fc5': 'b92ae025fbbdb31f.1605974236.1.1605975278.1605974236.',
    '_pk_ses.638735871.2fc5': '*',
    '_trackity': '70e316b0-96f2-dbe1-a2ed-43ff60419991',
    '_ga_NKX31X43RV': 'GS1.1.1605974235.1.1.1605975326.0',
    '_ga': 'GA1.1.657946765.1605974236',
    'ai_client_id': '11935756853.1605974227',
    'an_session': 'zizkzrzjzlzizqzlzqzjzdzizizqzgzmzkzmzlzrzmzgzdzizlzjzmzqzkznzhzhzkzdzizhzdzizlzjzmzqzkznzhzhzkzdzizlzjzmzqzkznzhzhzkzdzjzdzhzqzdzizd2f27zdzjzdzlzmzmznzq',
    'au_aid': '11935756853',
    'dgs': '1605975268%3A3%3A0',
    'au_gt': '1605974227146',
    '_ants_services': '%5B%22cuid%22%5D',
    '__admUTMtime': '1605974236',
    '__iid': '749',
    '__su': '0',
    '_bs': 'bb9a32f6-ab13-ce80-92d6-57fd3fd6e4c8',
    '_gid': 'GA1.2.867846791.1605974237',
    '_fbp': 'fb.1.1605974237134.1297408816',
    '_hjid': 'f152cf33-7323-4410-b9ae-79f6622ebc48',
    '_hjFirstSeen': '1',
    '_hjIncludedInPageviewSample': '1',
    '_hjAbsoluteSessionInProgress': '0',
    '_hjIncludedInSessionSample': '1',
    'tiki_client_id': '657946765.1605974236',
    '__gads': 'ID=ae56424189ecccbe-227eb8e1d6c400a8:T=1605974229:RT=1605974229:S=ALNI_MZFWYf2BAjzCSiRNLC3bKI-W_7YHA',
    'proxy_s_sv': '1605976041662',
    'TKSESSID': '8bcd49b02e1e16aa1cdb795c54d7b460',
    'TIKI_RECOMMENDATION': '21dd50e7f7c194df673ea3b717459249',
    'cto_bundle': 'V9Dkml9NVXNkQmJ6aEVLcXNqbHdjcVZoQ0l5RXpOMlRybjdDT0JrUnNxVXc2bHd0N2J3Y2FCdmZVQXdYY1QlMkJYUmxXSHZ2UEFwd3IzRHhzRWJYMlQlMkJsQjhjQlA1JTJCcExyRzlUQk5CYUdMdjl2STNQanVsa3cycHd3SHElMkJabnI3dzNhREpZcFVyandyM1d1QWpJbWYxT1UyWDdHZyUzRCUzRA',
    'TIKI_RECENTLYVIEWED': '58259141',
    '_ants_event_his': '%7B%22action%22%3A%22view%22%2C%22time%22%3A1605974691247%7D',
    '_gat': '1',
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'vi-VN,vi;q=0.8,en-US;q=0.5,en;q=0.3',
    'Referer': 'https://tiki.vn/?src=header_tiki',
    'x-guest-token': '8jWSuIDBb2NGVzr6hsUZXpkP1FRin7lY',
    'Connection': 'keep-alive',
    'TE': 'Trailers',
}

params = {
    'limit': '1',
    'include': 'comments,contribute_info,attribute_vote_summary',
    'sort': 'score%7Cdesc,id%7Cdesc,stars%7Call',
    'page': '1',
    'spid': '',
    'product_id': '',
    'seller_id': ''
}

In [3]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

filename = 'thoi_trang_nam'
extension = '.csv'

df = pd.read_csv(os.path.join(parent_dir, f'Data\{filename + extension}'))

df.head(1)

,id,sku,name,short_description,price,list_price,discount,discount_rate,review_count,rating_average,inventory_status,quantity_sold,brand_id,brand_name,spid,seller_id
0,50347611,8829610931441,Combo 2 Quần shorts thể thao nam dạng sọt spor...,Quần shorts thể thao nam dạng sọt sport đùiCh...,79000,130000,51000,39,165,4.5,available,552.0,224521,Doka,50348115,9691


In [4]:
def review_parser(review, product_id, seller_id, spid):
    d = {}
    try: 
        d['product_id'] = product_id
        d['seller_id'] = seller_id
        d['spid'] = spid
        
        data = review.get('data')[0]
        
        d['id'] = data['id']
        d['title'] = data['title']
        d['content'] = data['content']
        d['status'] = data['status']
        d['customer_id'] = data['customer_id']
        
        seller = data['seller']
        if seller:
            d['seller_id'] = seller.get('id')
            d['seller_name'] = seller.get('name')
        else:
            d['seller_id'] = None
            d['seller_name'] = None
            
        d['created_at'] = data['created_at']
        
        created_by = data['created_by']
        if created_by:
            d['name'] = created_by['name']
            d['full_name'] = created_by['full_name']
            d['purchased'] = created_by['purchased']
            d['purchased_at'] = created_by['purchased_at']
        else:
            d['name'] = None
            d['full_name'] = None
            d['purchased'] = None
            d['purchased_at'] = None
            
        # stars = review.get('stars')
        # if stars:
        #     d['1_star_count'] = stars.get('1').get('count')
        #     d['1_star_percent'] = stars.get('1').get('percent')
        #     d['2_star_count'] = stars.get('2').get('count')
        #     d['2_star_percent'] = stars.get('2').get('percent')
        #     d['3_star_count'] = stars.get('3').get('count')
        #     d['3_star_percent'] = stars.get('3').get('percent')
        #     d['4_star_count'] = stars.get('4').get('count')
        #     d['4_star_percent'] = stars.get('4').get('percent')
        #     d['5_star_count'] = stars.get('5').get('count')
        #     d['5_star_percent'] = stars.get('5').get('percent')
        # else:
        #     d['1_star_count'] = None
        #     d['1_star_percent'] = None
        #     d['2_star_count'] = None
        #     d['2_star_percent'] = None
        #     d['3_star_count'] = None
        #     d['3_star_percent'] = None
        #     d['4_star_count'] = None
        #     d['4_star_percent'] = None
        #     d['5_star_count'] = None
        #     d['5_star_percent'] = None
        
        # d['rating_average'] = review.get('rating_average')
        # d['reviews_count'] = review.get('reviews_count')
                
    except Exception as e:
        print(f"Error parsing reviews: {e}")
    return d

In [15]:
length = df['review_count'].sum()

progress_bar = tqdm(total=length, desc=f'Crawling reviews from {filename + extension}', position=0)

review_list = list()

for index, row in df.iterrows():
    # Đếm số lượng review cho sản phẩm đó
    review_count = row['review_count']
    
    # Lấy các giá trị id để tạo API
    params['product_id'] = row['id']
    params['seller_id'] = row['seller_id']
    params['spid'] = row['spid']
    
    limit = math.ceil(review_count / int(params['page']))
    
    for i in range(1, limit + 1):
        params['page'] = i
        response = requests.get('https://tiki.vn/api/v2/reviews', headers=headers, params=params, cookies=cookies)

        if response.status_code == 200:
            try:
                review = response.json()
                parsed_data = review_parser(review, params['product_id'], params['seller_id'], params['spid'])
                review_list.append(parsed_data)

            except json.JSONDecodeError as json_error:
                print(f'Error decoding JSON: {json_error}')
        else:
            print(f"Failed to crawl product {params['product_id']}. Status code: {response.status_code}")
        
        progress_bar.update(1)
    
progress_bar.close()


Crawling reviews from thoi_trang_nam.csv:   2%|▏         | 1095/66842 [05:51<4:10:42,  4.37it/s] 

Error parsing reviews: 'seller'


Crawling reviews from thoi_trang_nam.csv:   5%|▍         | 3273/66842 [17:19<4:00:08,  4.41it/s] 

Error parsing reviews: 'full_name'


Crawling reviews from thoi_trang_nam.csv:   5%|▍         | 3333/66842 [17:33<4:58:46,  3.54it/s]

...

Crawling reviews from thoi_trang_nam.csv:   5%|▌         | 3378/66842 [17:46<4:22:29,  4.03it/s]

Error parsing reviews: 'full_name'


Crawling reviews from thoi_trang_nam.csv:   5%|▌         | 3382/66842 [17:47<4:25:14,  3.99it/s]

Error parsing reviews: 'full_name'


Crawling reviews from thoi_trang_nam.csv:   9%|▉         | 6022/66842 [30:41<3:47:15,  4.46it/s] 

In [ ]:
save_name = filename + '_reviews' + extension

df_stars = pd.DataFrame(review_list)
df_stars.to_csv(os.path.join(parent_dir, f'Data\{save_name}'), index=False, encoding='utf-8')
print(f"Saved {save_name}")

Saved giay_dep_nu_stars.csv
